# Tool Pattern

In [1]:
from utils import (
    completions_create,
    build_prompt_structure,
    update_chat_history,
    ChatHistory,
    FixedFirstChatHistory,
    fancy_print,
    get_fn_signature,
    validate_arguments,
    Tool,
    tool,
    extract_tag_content,
    TagContentResult
)

In [2]:
from tools import hn_tool

In [3]:
hn_tool

In [4]:
hn_tool.name

'hn_tool'

In [5]:
import json
json.loads(hn_tool.fn_signature)

{'name': 'hn_tool',
 'description': '\n    Fetch the top stories from Hacker News.\n\n    This function retrieves the top `top_n` stories from Hacker News using the Hacker News API.\n    Each story contains the title, URL, score, author, and time of submission. The data is fetched\n    from the official Firebase Hacker News API, which returns story details in JSON format.\n\n    Args:\n        top_n (int): The number of top stories to retrieve.\n    ',
 'parameters': {'properties': {'top_n': {'type': 'int'}}}}

As you can see, the function signature has been automatically generated. It contains the `name`, a `description` (taken from the docstrings) and the `parameters`, whose types come from the typing annotations. Now that we have a tool, let's run the agent.

## Building the Tool-Using Agent

In [6]:
import re

from colorama import Fore
from openai import OpenAI
from dotenv import load_dotenv


# Load API key stored in .env
load_dotenv()

TOOL_SYSTEM_PROMPT = """
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call>
XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>,  "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools:

<tools>
%s
</tools>

If the user asks about something related to the tool, always provide the tool call.
Don't say that you can't access realtime information if you have a tool to achieve that.
"""


class ToolAgent:
    """
    The ToolAgent class represents an agent that can interact with a language model and use tools
    to assist with user queries. It generates function calls based on user input, validates arguments,
    and runs the respective tools.

    Attributes:
        tools (Tool | list[Tool]): A list of tools available to the agent.
        model (str): The model to be used for generating tool calls and responses.
        client (OpenAI): The OpenAI client used to interact with the language model.
        tools_dict (dict): A dictionary mapping tool names to their corresponding Tool objects.
    """

    def __init__(
        self,
        tools: Tool | list[Tool],
        model: str = "gpt-4o-mini",
    ) -> None:
        self.client = OpenAI()
        self.model = model
        self.tools = tools if isinstance(tools, list) else [tools]
        self.tools_dict = {tool.name: tool for tool in self.tools}

    
    def add_tool_signatures(self) -> str:
        """
        Collects the function signatures of all available tools.

        Returns:
            str: A concatenated string of all tool function signatures in JSON format.
        """
        return "".join([tool.fn_signature for tool in self.tools])

        
    def process_tool_calls(self, tool_calls_content: list) -> dict:
        """
        Processes each tool call, validates arguments, executes the tools, and collects results.

        Args:
            tool_calls_content (list): List of strings, each representing a tool call in JSON format.

        Returns:
            dict: A dictionary where the keys are tool call IDs and values are the results from the tools.
        """
        observations = {}
        for tool_call_str in tool_calls_content:
            tool_call = json.loads(tool_call_str)
            tool_name = tool_call["name"]
            tool = self.tools_dict[tool_name]

            print(Fore.GREEN + f"\nUsing Tool: {tool_name}")

            # Validate and execute the tool call
            validated_tool_call = validate_arguments(
                tool_call, json.loads(tool.fn_signature)
            )
            print(Fore.GREEN + f"\nTool call dict: \n{validated_tool_call}")

            result = tool.run(**validated_tool_call["arguments"])
            print(Fore.GREEN + f"\nTool result: \n{result}")

            # Store the result using the tool call ID
            observations[validated_tool_call["id"]] = result

        return observations

    
    def run(
        self,
        user_msg: str,
    ) -> str:
        """
        Handles the full process of interacting with the language model and executing a tool based on user input.

        Args:
            user_msg (str): The user's message that prompts the tool agent to act.

        Returns:
            str: The final output after executing the tool and generating a response from the model.
        """
        user_prompt = build_prompt_structure(prompt=user_msg, role="user")

        tool_chat_history = ChatHistory(
            [
                build_prompt_structure(
                    prompt=TOOL_SYSTEM_PROMPT % self.add_tool_signatures(),
                    role="system",
                ),
                user_prompt,
            ]
        )
        agent_chat_history = ChatHistory([user_prompt])

        tool_call_response = completions_create(
            self.client, messages=tool_chat_history, model=self.model
        )
        tool_calls = extract_tag_content(str(tool_call_response), "tool_call")

        if tool_calls.found:
            observations = self.process_tool_calls(tool_calls.content)
            update_chat_history(
                agent_chat_history, f'f"Observation: {observations}"', "user"
            )

        return completions_create(self.client, agent_chat_history, self.model)


In [7]:
tool_agent = ToolAgent(tools=[hn_tool])

A quick check to see that everything works fine. If we ask the agent something unrelated to Hacker News, it shouldn't use the tool.

In [8]:
output = tool_agent.run(user_msg="Tell me your name")

In [9]:
fancy_print(output)


I’m called Assistant. How can I help you today?



Now, let's ask for specific information about Hacker News.

In [10]:
output = tool_agent.run(user_msg="Tell me the top 5 Hacker News stories")


Using Tool: hn_tool

Tool call dict: 
{'name': 'hn_tool', 'arguments': {'top_n': 5}, 'id': 1}

Tool result: 
[{"title": "Tell HN: Help restore the tax deduction for software dev in the US (Section 174)", "url": "No URL available"}, {"title": "Containerization is a Swift package for running Linux containers on macOS", "url": "https://github.com/apple/containerization"}, {"title": "The Danish Ministry of Digitalization Is Switching to Linux and LibreOffice", "url": "https://politiken.dk/viden/tech/art10437680/Caroline-Stage-udfaser-Microsoft-i-Digitaliseringsministeriet"}, {"title": "Apple announces Foundation Models and Containerization frameworks, etc", "url": "https://www.apple.com/newsroom/2025/06/apple-supercharges-its-tools-and-technologies-for-developers/"}, {"title": "Show HN: A MCP server and client implementing the latest spec", "url": "https://github.com/hemanth/paws-on-mcp"}]


InternalServerError: Error code: 500 - {'error': {'message': 'The server had an error while processing your request. Sorry about that!', 'type': 'server_error', 'param': None, 'code': None}}

In [ ]:
fancy_print(output)

There you have it!! A fully functional Tool!! 🛠️